<a href="https://colab.research.google.com/github/SanchayRohad/Data-Engineering-Job-Trends-Analysis/blob/main/WebScraping-Linkedin-DE-Jobs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import urllib
from datetime import datetime
import pandas as pd
import time
import logging
from google.colab import files
from requests.packages.urllib3.exceptions import InsecureRequestWarning

In [2]:
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
# response = requests.get(url, headers=headers, verify=False)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
titles={
    "Data Engineer",
    "Database Engineer",
    "Data Architect",
    "ETL Developer",
    "Data Integration Engineer",
    "Big Data Engineer",
    "Data Warehouse Engineer",
    "Data Modeler",
    "Data Pipeline Engineer",
    "Data and AI Engineer",
    "Software Engineer, Data",
    "DataOps Engineer"
}

In [4]:
locations = {
  'Bangalore',
  'Pune',
  'Mumbai',
  'Kolkata',
  'Hyderabad',
  'Chennai',
  'Nagpur',
  'Delhi',
  'Gurgaon',
  'Noida',
  'Kochi'
}

In [14]:
def scrape_jobs(job_title, location, num_of_pages, applied_jobs_list):
  info = []
  headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'};

  for page in range(num_of_pages):
    getVars = {'keywords': job_title, 'location': location,'sort':'date', 'start':str(page*10), 'f_TPR':'r604800'}
    url = 'https://www.linkedin.com/jobs/search?' + urllib.parse.urlencode(getVars)
    response = requests.get(url, headers=headers, verify = False)
    soup = BeautifulSoup(response.content, 'html.parser')
    jobs = soup.find_all('div', class_='base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card')
    for job in jobs:
      title = str(job.find('h3', class_='base-search-card__title').text.strip())
      company = str(job.find('h4', class_='base-search-card__subtitle').text.strip())
      post_date = datetime.strptime(job.find('time')['datetime'], '%Y-%m-%d').date()
      print(job)

      try:
        link = job.find('a', class_='base-card__full-link')['href'].split('?')[0]
        inp = link.split('/')[-1]
        inp = ''.join(' ' if not ch.isdigit() else ch for ch in inp).strip()
        id = inp.split()[-1]
        link = 'https://www.linkedin.com/jobs/view/' + str(id)
        location_text = str(job.find('span', class_='job-search-card__location').text.strip())
        info.append([title, company, post_date, link, location_text])
        print(info)

      except :
        print('error parsing')

  cols = ['Job_title', 'Company', 'Job_post_date', 'Link', 'Location']

  info_table = pd.DataFrame(info, columns=cols)
  info_table = info_table.drop_duplicates(subset=['Job_title','Company'])
  info_table = info_table[~info_table['Job_title'].isin(applied_jobs_list)]

  info_table = info_table.sort_values(by='Job_post_date', ascending=False)
  info_table = info_table.reset_index(drop=True)
  return info_table
all_jobs = []
for location in locations:
    for job_title in titles:
        num_of_pages = 1
        applied_jobs_list = []

        logging.info(f"Scraping job listings for {job_title} in {location}...")
        job_df = scrape_jobs(job_title, location, num_of_pages, applied_jobs_list)
        if not job_df.empty:
            # Append the DataFrame of the current iteration to the all_jobs list
            all_jobs.append(job_df)

# If there is any scraped job data, concatenate all DataFrames into a single DataFrame
if all_jobs:
    print(all_jobs)
    final_df = pd.concat(all_jobs, ignore_index=True)

    # Creating a unique filename based on the current date
    file_name = f"all_scraped_jobs_{datetime.now().strftime('%Y%m%d')}.csv"

    # Save the final DataFrame to CSV
    final_df.to_csv(file_name, index=False)

    # Log that the file was saved
    logging.info(f"All job data saved to {file_name}")

    # Trigger download after saving the file
    files.download(file_name)
else:
    logging.warning("No job data found.")

Streaming output truncated to the last 5000 lines.
28  https://www.linkedin.com/jobs/view/4114260293  Hyderabad, Telangana, India  
29  https://www.linkedin.com/jobs/view/4114263236  Hyderabad, Telangana, India  
30  https://www.linkedin.com/jobs/view/4114251210  Hyderabad, Telangana, India  
31  https://www.linkedin.com/jobs/view/4114251463  Hyderabad, Telangana, India  
32  https://www.linkedin.com/jobs/view/4114261498  Hyderabad, Telangana, India  
33  https://www.linkedin.com/jobs/view/4114259575  Hyderabad, Telangana, India  
34  https://www.linkedin.com/jobs/view/4114255783  Hyderabad, Telangana, India  
35  https://www.linkedin.com/jobs/view/4114257921  Hyderabad, Telangana, India  
36  https://www.linkedin.com/jobs/view/4114261736  Hyderabad, Telangana, India  
37  https://www.linkedin.com/jobs/view/4114202514  Hyderabad, Telangana, India  
38  https://www.linkedin.com/jobs/view/4114257064  Hyderabad, Telangana, India  
39  https://www.linkedin.com/jobs/view/4113899645  Hyderab

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
 # LinkedIn wont allow to webscrape this easily !! due to security

def job_descriptions(info_table, CV_Clear):
  # Finding match percentage
  headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'};
  match_percents = []
  image_links = []
  locations = []
  job_descriptions =[]
  salary_ranges = []
  for l in info_table['Link']:
    try:
      response = requests.get(l, headers=headers)
      soup = BeautifulSoup(response.content, 'html.parser')

      job_description = str(soup.find('div', class_='description__text description__text--rich').find('div'))

      salary_range = extract_currency_values(job_description.replace(',',''))
      if len(salary_range)>0:
        salary_range = '$'+salary_range[0] + '- $' + salary_range[1]
      else:
        salary_range = 'No Salary Info Found'

      image_raw_links = soup.find('div', class_='description__text description__text--rich').find_all('img')
      image_link = image_raw_links.replace('amp;','')

      location = soup.find('span', class_='topcard__flavor topcard__flavor--bullet').text.strip()
      locations.append(location)

      Match_Test =[CV_Clear, job_description]
      cv = CountVectorizer()
      count_matrix = cv.fit_transform(Match_Test)
      MatchPercentage = cosine_similarity(count_matrix)
      match_percents.append(MatchPercentage[0][1]*100)

      job_descriptions.append(job_description)
      salary_ranges.append(salary_range)
      image_links.append(image_link)
    except:
      print(f"Error fetching job details: {e}")
      job_descriptions_list.append("Error fetching description")
      match_percents.append(0)
      salary_ranges.append("No salary info")

    info_table['Job_Description'] = job_descriptions_list
    info_table['Match_Percentage'] = match_percents
    info_table['Salary_Range'] = salary_ranges
    return info_table